# Tracking

## 概述

使用类继承的方法，无需修改自动生成的文件。

```shell
root f8ppac001.root

tree->MakeClass("trackingBase");
```

将生成的类命名为`trackingBase`，意为`Tracking`的基类，以此为基础编写`Tracking`类

## Tracking 类

### Tracking.h
```c++
// Tracking.h
#ifndef Tracking_h
#define Tracking_h

#include "trackingBase.h"
#include <TROOT.h>
#include <TChain.h>
#include <TFile.h>
#include <TGraph.h>
#include <TF1.h>
#include <TH2D.h>
#include <TFitResult.h>

class Tracking: public trackingBase {
public :
	Tracking(TTree *tree = 0);
	virtual ~Tracking();
	virtual void Loop(TTree *tree);
private:
    // anode
    Double_t anode[5];
    // variable for x tracking
    Double_t xx[5], xz[5];
    // variable for y tracking
    Double_t yy[5], yz[5];
    // residual
    Double_t dx[5], dy[5];
    // target position
    Double_t tx, ty, tz;
    // target position after projection
    Double_t ptx, pty;
    // chi2/ndf
    Double_t c2nx, c2ny;
    // number of PPAC to track, 0 for tracking failure
    // 用于重建的探测器数量，0表示不满足重建条件
    Int_t numX, numY;
    // combination of the valid PPAC,用于重建径迹的探测器组合
    // 使用标志位的方式记录
    Int_t trackX, trackY;
    // 拟合得到的径迹方程
    Double_t bx, kx;
    Double_t by, ky;
    // 重建后的径迹在不同探测器上的位置
    Double_t fxx[5], fyy[5];

    // init the variables, return: 0 -- can't track, 1 -- track x, 2 -- track y, 3 -- track x and y
    virtual Int_t trackInit();
    virtual void setBranch(TTree *tree);
    virtual void addTrace(TH2D *h, Double_t k, Double_t b, Int_t minBin, Int_t maxBin);
    virtual Double_t simpleFit(TGraph *g, Double_t &k, Double_t &b);
};

#endif
```

### Tracking.cpp
```c++
// Tracking.cpp
#include "Tracking.h"

Tracking::Tracking(TTree *tree)
	: trackingBase(tree) {
}


Tracking::~Tracking() {
}



void Tracking::setBranch(TTree *tree) {
	tree->Branch("xx", &xx, "xx[5]/D");
	tree->Branch("xz", &xz, "xz[5]/D");
	tree->Branch("yy", &yy, "yy[5]/D");
	tree->Branch("yz", &yz, "yz[5]/D");
	tree->Branch("anode", &anode, "anode[5]/D");

	tree->Branch("dx", &dx, "dx[5]/D");
	tree->Branch("dy", &dy, "dy[5]/D");

	tree->Branch("tx", &tx, "tx/D");
	tree->Branch("ty", &ty, "ty/D");
	tree->Branch("tz", &tz, "tz/D");

	tree->Branch("ptx", &ptx, "ptx/D");
	tree->Branch("pty", &pty, "pty/D");

	tree->Branch("c2nx", &c2nx, "c2nx/D");
	tree->Branch("c2ny", &c2ny, "c2ny/D");

	tree->Branch("beamTrig", &beamTrig, "beamTrig/I");
	tree->Branch("must2Trig", &must2Trig, "must2Trig/I");

	tree->Branch("targetX", &targetX, "targetX");
	tree->Branch("targetY", &targetY, "targetY");

	tree->Branch("numX", &numX, "numX/I");
	tree->Branch("numY", &numY, "numY/I");

	tree->Branch("trackX", &trackX, "trackX/I");
	tree->Branch("trackY", &trackY, "trackY/I");

	tree->Branch("bx", &bx, "bx/D");
	tree->Branch("kx", &kx, "kx/D");
	tree->Branch("by", &by, "by/D");
	tree->Branch("ky", &ky, "ky/D");

	tree->Branch("fxx", &fxx, "fxx[5]/D");
	tree->Branch("fyy", &fyy, "fyy[5]/D");
}


Int_t Tracking::trackInit() {
	tx = -999;
	ty = -999;

	for (int i = 0; i != 5; ++i) {
		xx[i] = PPACF8[i][0];
		xz[i] = PPACF8[i][2];
		yy[i] = PPACF8[i][1];
		yz[i] = PPACF8[i][3];
		anode[i] = PPACF8[i][4];
	}

	// 判断是否满足重建流程：
	// 	1. 遍历所有探测器，记录有信号的探测器总数量和组合
	//	2. 如果数量等于1，不要
	//	3. 如果数量为2，且所有信号的都在PPAC1或者PPAC2里面，不要
	//	4. 其他都要

	// check x
	numX = 0;
	trackX = 0;
	for (int i = 0; i != 4; ++i) {
		if (abs(xx[i]) < 120) {
			numX += 1;
			trackX |= 1 << i;
		}
	}
	// PPAC3
	if (abs(xx[4]) < 50) {
		numX += 1;
		trackX |= 1 << 4;
	}

	if (numX == 1) numX = -1;
	if (numX == 2) {
		if (trackX & 0x3) numX = -2;			// 信号都在PPAC1
		if (trackX & 0xC) numX = -2;			// 信号都在PPAC2
	}


	// check y
	numY = 0;
	trackY = 0;
	for (int i = 0; i != 4; ++i) {
		if (abs(yy[i]) < 75) {
			numY += 1;
			trackY |= 1 << i;
		}
	}
	// PPAC3
	if (abs(yy[4]) < 50) {
		numY += 1;
		trackY |= 1 << 4;
	}

	if (numY == 1) numY = -1;
	if (numY == 2) {
		if (trackY & 0x3) numY = -2;
		if (trackY & 0xC) numY = -2;
	}


	Int_t flag = 0;
	if (numX > 0) flag |= 1;
	if (numY > 0) flag |= 2;
	return flag;
}


void Tracking::addTrace(TH2D *h, Double_t k, Double_t b, Int_t minBin, Int_t maxBin) {
	if (h == nullptr) return;
	if (minBin >= maxBin) return;

	for (int i = minBin; i != maxBin; i+=10) {
		h->Fill(i, (Int_t)(i*k*b));
	}
	return;
}


void Tracking::Loop(TTree *tree) {
	TH2D *htf8xz = new TH2D("htf8xz", "xz track by ppac", 220, -2000, 200, 300, -150, 150);
	TH2D *htf8yz = new TH2D("htf8yz", "yz track by ppac", 220, -2000, 200, 300, -150, 150);

	// TFile *opf = new TFile("../data/tracking.root", "recreate");
	// TTree *tree = new TTree("tree", "ppac tracking");
	setBranch(tree);



	if (fChain == 0) return;

	Long64_t nentries = fChain->GetEntriesFast();
	Long64_t nbytes = 0, nb = 0;
	for (Long64_t jentry = 0; jentry != nentries; jentry++) {
		// load data
		Long64_t ientry = LoadTree(jentry);
		if (ientry < 0) break;
		nb = fChain->GetEntry(jentry);
		nbytes += nb;



		// x and y tracked
		if (trackInit() != 3) continue;



		// fit part
		TGraph *grx = new TGraph;
		int points = 0;
		for (int i = 0; i != 5; ++i) {
			if (trackX & 1<<i) {
				grx->SetPoint(points, xz[i], xx[i]);
				points++;
			}
		}
		if (sFit) {
			c2nx = simpleFit(grx, kx, bx);
			c2nx = points == 2 ? 0.0 : c2nx / (points-2);
			for (int i = 0; i != 5; ++i) {
				fxx[i] = kx * xz[i] + bx;
				dx[i] = (trackX & 1<<i) ? xx[i] - fxx[i] : -999;
			}
		} else {
			TF1 *fx = new TF1("fx", "pol1", -2000, 100);
			TFitResultPtr r = grx->Fit(fx, "SQ");
			bx = fx->GetParameter(0);
			kx = fx->GetParameter(1);
			// residual and fitted position
			for (int i = 0; i != 5; ++i) {
				fxx[i] = fx->Eval(xz[i]);
				dx[i] = (trackX & 1<<i) ? xx[i] - fxx[i] : -999;
			}
			c2nx = r->Chi2() / r->Ndf();
			delete fx;
		}
		delete grx;

		TGraph* gry = new TGraph;
		points = 0;
		for (int i = 0; i != 5; ++i) {
			if (trackY & 1<<i) {
				gry->SetPoint(points, yz[i], yy[i]);
				points++;
			}
		}
		TF1 *fy = new TF1("fy", "pol1", -2000, 100);
		TFitResultPtr r = gry->Fit(fy, "SQ");
		by = fy->GetParameter(0);
		ky = fy->GetParameter(1);
		// residual
		for (int i = 0; i != 5; ++i) {
			fyy[i] = fy->Eval(yz[i]);
			dy[i] = (trackY & 1<<i) ? yy[i] - fyy[i] : -999;
		}
		c2ny = r->Chi2() / r->Ndf();
		delete fy;
		delete gry;

		fitt += clock() - t;

		// position part
		// target position
		tz = -bx / (kx + 1.0);
		tx = -tz;
		tx = bx;
		ty = ky * tz + by;
		// projection
		ptx = tx * 1.4142;			// tx / sqrt(2.0)
		pty = ty;



		// add trace part
		// set trace
		addTrace(htf8xz, kx, bx, -1800, 100);
		addTrace(htf8yz, ky, by, -1800, 100);



		// fill part
		tree->Fill();



		if (jentry % 100000 == 0) std::cout << jentry << "/" << nentries << std::endl;
	}


	htf8xz->Write();
	htf8yz->Write();

}

```
### 逻辑
#### 变量
+ 在不同探测器上的信号 `anode[5], xx[5], xz[5], yy[5], yz[5]`
+ 径迹的方程 `bx, kx, by, ky`
+ 径迹在探测器上的位置 `fxx[5], fyy[5]`
+ 残差 `dx[5], dy[5]`，平方差 `c2nx, c2ny`
+ 靶上坐标 `tx, ty, tz`，投影后的坐标`ptx, pty`
+ 重建径迹的探测器数量`numX, numY`
+ 探测器组合，使用标志位方式记录`trackX, trackY`

#### 方法
+ `setBranch` 初始化输出的树
+ `trackInit` 初始化变量，判断事件是否满足重建要求，流程如下
    1. 遍历所有探测器，记录有信号的探测器总数量和组合
    2. 如果数量等于1，不要
    3. 如果数量为2，且所有信号的都在PPAC1或者PPAC2里面，不要
    4. 其他都要
+ `addTrace` 经径迹添加到图里
+ `Loop` 主要tracking方法
    1. load getEntry
    2. track init
    3. fit
    4. calculate target position
    5. add trace
    6. fill


## main

```c++
// main.cpp
#include <cstring>
#include <iostream>
#include <TFile.h>
#include <TString.h>
#include <TTree.h>
#include "Tracking.h"

int main(int argc, char **argv) {
	if (argc != 1) {
		std::cout << "usage: ./Tracking" << std::endl;
	}

	TString inFile("../data/f8ppac001.root");
	TFile *ipf = new TFile(inFile.Data());
	if (!ipf->IsOpen()) {
		std::cout << "Error open file " << inFile << std::endl;
	}
	TTree *ipt = (TTree*)ipf->Get("tree");

	TFile *opf;
	TTree *opt;

	opf = new TFile("../data/tracking.root", "recreate");
	opt = new TTree("tree", "ppac tracking");
	Tracking *tk = new Tracking(ipt);
	tk->Loop(opt);
	opt->Write();
	opf->Close();



	ipf->Close();

	return 0;
}

```


## Makefile

```Makefile
GXX = g++
OBJS = main.o Tracking.o trackingBase.o
SRC = main.cpp Tracking.cpp trackingBase.cpp

ROOTCFLAGS = $(shell root-config --cflags)
INCLUDE = -I ../include
CFLAGS = -Wall -O3 $(ROOTCFLAGS) $(INCLUDE)

ROOTLIBS = $(shell root-config --libs)
LIBS = $(ROOTLIBS)
LDFLAGS = $(LIBS)


all:
	make Tracking

Tracking: $(OBJS)
	$(GXX) -o $@ $^ $(LDFLAGS)
$(OBJS):%.o:%.cpp
	$(GXX) $(CFLAGS) -c $<


clean:
	rm *.o Tracking


```